남은 거

TTS API

영상 + 오디오



# 순서도

1. 이미지 경로 설정

2. 오디오 길이 측정

3. vfx 설정(각 함수에 맞게 오디오 설정)

4. 전환 효과 설정


결과물 shape

자막 : 30

장면 프롬프트 : 30


In [2]:
!pip install moviepy

     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     --------- ----------------------------- 92.2/388.3 kB 2.6 MB/s eta 0:00:01
     -------------------------------------  378.9/388.3 kB 4.7 MB/s eta 0:00:01
     -------------------------------------- 388.3/388.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/313.4 kB ? eta -:--:--
   ---------------------------------------- 313.4/313.4 kB 9.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/22.6 MB ? eta -:--:--
   - -------------------------------------- 0.7/22.6 MB 14.8 MB/s eta 0:00:02
   -- ------------------------------------- 1.4/22.6 MB 15


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\fhsdn_6m\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import glob, cv2, random, tqdm, src, moviepy, os
import numpy as np
from moviepy.editor import AudioFileClip, concatenate_videoclips, ImageSequenceClip, concatenate_audioclips, CompositeAudioClip

In [2]:
root_path = 'D:/SKT'

In [8]:
subtitle = [['흙수저 출신 윤현우,', '순양그룹에 발을 들여놓다.'],
 ['영화같은 인생을 뒤흔든', '비극적 총성.'],
 ['죽음에서 깨어,', '재벌집 막내아들로.'],
 ['진도준, 잊힌 막내아들에서', '순양그룹의 야망가로.'],
 ['할아버지 진양철의 도전,', '한 아이의 운명을 바꾸다.'],
 ['8만 평의 목장,', '소년의 첫 승리.'],
 ['10살의 천재,', '구글에 투자하다.'],
 ['작은 투자가', '몇 조의 자산으로.'],
 ['미라클 인베스트먼트,', '숨겨진 대마법사의 탄생.'],
 ['부실기업을 덫으로,', '순양그룹을 향한 진도준의 복수.'],
 ['미라클 인베스트먼트,', '기업들을 하나둘씩 삼키다.'],
 ['그림자에서 지휘하는', '복수의 오케스트라.'],
 ['순양그룹을 삼킨 자,', '진도준의 성공.'],
 ['과거의 적들과 마주한 순간,', '복수의 결말.'],
 ['재벌 막내아들에서', '순양그룹의 주인으로.'],
 ['더 자세한 내용은?',"'재벌집 막내아들' 에서!"]]

effects = ['페이드 인',
 '눈 깜빡임 효과',
 '페이드 아웃',
 '줌 인',
 '줌 아웃',
 '페이드 인',
 '눈 깜빡임 효과',
 '배경 없이 나레이션만',
 '줌 아웃',
 '페이드 아웃',
 '줌 인',
 '페이드 인',
 '줌 아웃',
 '눈 깜빡임 효과',
 '배경 없이 나레이션만']

In [9]:
selected_image = [0,1,1,0,0,1,1,0,1,0,0,1,0,0,0]
img_path = []
for i,j in enumerate(selected_image):
    filename= f"{root_path}/image/{i:02d}-{j:02d}.png"
    assert os.path.exists(filename)
    img_path.append(filename)
img_path

['D:/SKT/image/00-00.png',
 'D:/SKT/image/01-01.png',
 'D:/SKT/image/02-01.png',
 'D:/SKT/image/03-00.png',
 'D:/SKT/image/04-00.png',
 'D:/SKT/image/05-01.png',
 'D:/SKT/image/06-01.png',
 'D:/SKT/image/07-00.png',
 'D:/SKT/image/08-01.png',
 'D:/SKT/image/09-00.png',
 'D:/SKT/image/10-00.png',
 'D:/SKT/image/11-01.png',
 'D:/SKT/image/12-00.png',
 'D:/SKT/image/13-00.png',
 'D:/SKT/image/14-00.png']

In [12]:
audio_path = sorted(glob.glob(f"{root_path}/audio/*"))

In [13]:
bgm = AudioFileClip(f"{root_path}/BGM.wav")

In [14]:


def floor(lst:list):
    for i in range(len(lst)):
        for j in range(len(lst[i])):
            lst[i][j] = int(lst[i][j])
    return lst


fps = 60 # 영상 프레임

imgs = [] # 이미지
audios = [] # 오디오

audio_frames = [] # 오디오별 길이

for p in img_path:
    imgs.append(cv2.cvtColor(cv2.imread(p),cv2.COLOR_BGR2RGB))
imgs.append(np.zeros_like(imgs[-1],dtype=np.uint8))

idx = 0 
for i in range(len(subtitle)):
    audios.append([])
    audio_frames.append([])
    for j in range(len(subtitle[i])):
        audios[i].append(AudioFileClip(audio_path[idx]))
        audio_frames[i].append(audios[i][j].duration*fps)
        idx = idx + 1

audio_frames = floor(audio_frames)

print(len(imgs),len(audios))
transition_frames = [30,30]
transition_lst = [src.transition.Slide,src.transition.Blending]
transition = []

frames = [sum(audio_frames[0])] # 
for i in range(1,len(imgs)):
    idx = random.randint(0,len(transition_lst)-1)
    transition.append(transition_lst[idx](transition_frames[idx]))
    frames.append(transition_frames[idx])
    frames.append(sum(audio_frames[i]))
frames

16 16


[224,
 30,
 174,
 30,
 143,
 30,
 230,
 30,
 188,
 30,
 165,
 30,
 133,
 30,
 135,
 30,
 219,
 30,
 211,
 30,
 218,
 30,
 179,
 30,
 179,
 30,
 194,
 30,
 174,
 30,
 167]

In [17]:
vfx_lst = [
    "없음",
    "페이드 인",
    "페이드 아웃",
    "줌 인",
    "줌 아웃",
    "눈 깜빡임 효과",
    "배경 없이 나레이션만"
]
def identity(video:src.Video):
    return video

vfx = [
    identity,
    src.effect.Fade_in(0,60),
    src.effect.Fade_out(-60),
    src.effect.Zoom_in(0),
    src.effect.Zoom_out(0),
    src.effect.Blink(0,60),
    identity
]
vfx_idx = []
for j,e in enumerate(effects):
    for i in range(len(vfx_lst)):
        if e.find(vfx_lst[i]) != -1:
            vfx_idx.append(i)
            break
vfx_idx.append(6)
vfx_idx,len(vfx_idx)

([1, 5, 2, 3, 4, 1, 5, 6, 4, 2, 3, 1, 4, 5, 6, 6], 16)

In [18]:
def apply_subtitle(video:src.Video,idx:int):
    start_frame = 0
    ret = video
    if vfx_idx[idx] == 6:
        for i in range(len(subtitle[idx])):
            ret = src.effect.Narration(subtitle[idx][i],start_frame=start_frame,end_frame=start_frame+audio_frames[idx][i])(ret)
            start_frame = start_frame + audio_frames[idx][i]
    else:
        for i in range(len(subtitle[idx])):
            ret = src.effect.Subtitle(subtitle[idx][i],start_frame=start_frame,end_frame=start_frame+audio_frames[idx][i],color=(255,255,255),bg_color=(0,0,0))(ret)
            start_frame = start_frame + audio_frames[idx][i]
    return ret

videos = [apply_subtitle(vfx[vfx_idx[0]](src.im2vid(imgs[0],sum(audio_frames[0]))),0)]
for i in tqdm.tqdm(range(1,len(imgs))):
    x1 = videos[-1]
    x2 = apply_subtitle(vfx[vfx_idx[i]](src.im2vid(imgs[i],sum(audio_frames[i]))),i)
    videos.append(transition[i-1](x1,x2))
    videos.append(x2)

In [ ]:
videos

[Frame shape(h,w):(1792, 1024), Num_frames:224, FPS:60, Duration:3.7333333333333334 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:174, FPS:60, Duration:2.9 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:143, FPS:60, Duration:2.3833333333333333 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:230, FPS:60, Duration:3.8333333333333335 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:188, FPS:60, Duration:3.1333333333333333 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:165, FPS:60, Duration:2.75 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:133, FPS:60, D

In [ ]:
video_lst = []


for i in tqdm.tqdm(range(len(videos))):
    vid = ImageSequenceClip(videos[i].video, fps=fps)
    if i != len(videos):
        if i%2 == 0:
            video_lst.append(vid.set_audio(concatenate_audioclips(audios[i//2])))
        else :
            video_lst.append(vid)
    
video_clip = concatenate_videoclips(video_lst)
new_audio = CompositeAudioClip([video_clip.audio,bgm.audio_loop(duration=video_clip.duration).volumex(1.5)])
video_clip.audio = new_audio


# 비디오를 저장합니다.
video_clip.write_videofile("test.mp4")

100%|██████████| 29/29 [00:00<00:00, 517.85it/s]


Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
